<a href="https://colab.research.google.com/github/Kaivan-Khazeni/Machine_Learning/blob/main/HW1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import io
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import numpy as np
import statsmodels.formula.api as smf
import seaborn as sb

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
labels_value = ['unacc','acc','good','vgood']
#label is where the values of each item is stored.  It is not binary,
#it is unacceptable, acceptable, good, very good.  This is different 
#then before
attributes = ['buying','maint','doors','persons','lug_boot','safety','label']

In [41]:
df = pd.read_csv ('train.csv',names=attributes)
df.head(5)
df_test = pd.read_csv ('test.csv',names=attributes)
df_test.head(5)

,buying,maint,doors,persons,lug_boot,safety,label
0,vhigh,high,5more,2,small,low,unacc
1,low,low,5more,2,small,med,unacc
2,low,vhigh,4,2,med,low,unacc
3,high,vhigh,3,4,med,med,unacc
4,vhigh,low,4,4,med,low,unacc


In [5]:
test_dict = {'Outlook':['S','S','O','R','R','R','O','S','S','R','S','O','O','R'],
         'Temp':['H','H','H','M','C','C','C','M','C','M','M','M','H','M'],
         'Humid':['H','H','H','H','N','N','N','H','N','N','N','H','N','H'],
         'Wind':['W','S','W','W','W','S','S','W','W','W','S','S','W','S'],
         'Play':['-','-','+','+','+','-','+','-','+','+','+','+','+','-']}
test_df = pd.DataFrame(test_dict)
test_df

,Outlook,Temp,Humid,Wind,Play
0,S,H,H,W,-
1,S,H,H,S,-
2,O,H,H,W,+
3,R,M,H,W,+
4,R,C,N,W,+
5,R,C,N,S,-
6,O,C,N,S,+
7,S,M,H,W,-
8,S,C,N,W,+
9,R,M,N,W,+


In [6]:
dummy_df = df[["buying" ,"label"]]

dummy_df['buying'].unique()


array(['low', 'vhigh', 'high', 'med'], dtype=object)

In [7]:
dummy_low = dummy_df[df.buying =='med']
print(len(dummy_low))
len(dummy_low[dummy_low.label == "good"])

248


17

In [8]:
dummy_low.label.unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [9]:
test_dictB = {'A':['K','A','I','K','A'],
         'B':['A','A','C','B','B'],
         'Y':['yes','no','so','maybe','so']}
test_dfB = pd.DataFrame(test_dictB)
test_dfB['A']

0    K
1    A
2    I
3    K
4    A
Name: A, dtype: object

In [10]:
attribute_values = {
    "buying": df['buying'].unique(),
    "maint": df['maint'].unique(),
    "doors": df['doors'].unique(),
    "persons": df['persons'].unique(),
    "lug_boot": df['lug_boot'].unique(),
    "safety": df['safety'].unique()
}

In [11]:
hey = test_df[['Outlook',test_df.keys()[-1]]]
hey[hey["Outlook"] == "S"]

,Outlook,Play
0,S,-
1,S,-
7,S,-
8,S,+
10,S,+


In [39]:
decision_table = ID3(-1*float("inf"),1,1,df,"none",tree=None)
decision_table

{'buying': 0.1015247071248564, 'maint': 0.07741985577459665, 'doors': 0.006726514230977809, 'persons': 0.22441128678577127, 'lug_boot': 0.03688725199484155, 'safety': 0.25822501448993584}
{'buying': 0.1496223368823797, 'maint': 0.13639630335579112, 'doors': 0.012104212356617072, 'persons': 0.3238309442054772, 'lug_boot': 0.09072212234577748}
{'buying': 0.2149705218635023, 'maint': 0.13719227312353488, 'doors': 0.018366645266694848, 'persons': 0.511350583869971, 'lug_boot': 0.12600833155954394}


{'safety': {'high': {'persons': {'2': 'unacc', '4': 'acc', 'more': 'acc'}},
  'low': 'unacc',
  'med': {'persons': {'2': 'unacc', '4': 'acc', 'more': 'acc'}}}}

In [40]:
import pprint

pprint.pprint(decision_table)

{'safety': {'high': {'persons': {'2': 'unacc', '4': 'acc', 'more': 'acc'}},
            'low': 'unacc',
            'med': {'persons': {'2': 'unacc', '4': 'acc', 'more': 'acc'}}}}


ID3 function below will take in a depth parameter and a variant indicator.  This way, we can specify if we want
to use Entropy, ME, or Gini Index.  It will also take in the S table.

In [43]:
#max depth will range from 1 - 6
def ID3(max_labels,max_depth, variant,S,A,tree=None):

  #if len(S[S.keys()[-1]].unique()) > max_labels:
  max_labels = len(S[S.keys()[-1]].unique())
  labels_value = S[S.keys()[-1]].unique()
  Class = S.keys()[-1]
  #Step 1: check if all examplesa have the same label
  #keep counter to make sure iterations do not exceed the max depth
  if max_depth >= 0:
    
    #print("In this call 1, 2 ..")
    #Finding root based off of gain
    #A. Find Total entropy using variant
    total_entropy = find_total_entropy(max_labels, variant,S)
    #print("Found total entropy 1,2")
    #print(total_entropy)
    #B. Now find the best split, which is A.
    table_attributes = S.loc[:,S.columns != S.keys()[-1]].columns
    #print(table_attributes)

    best_A = find_best_split(max_labels,variant,table_attributes,S,total_entropy)
    #print(best_A)
   
    #return best_A
    A = sorted(best_A.items(), key=lambda x: x[1],reverse=True)[0][0]


    #print(A)
    if tree is None:                    
      tree={}
      tree[A] = {}

   
    values_of_A = S[A].unique()

    for A_val in values_of_A:
      Sv = S.loc[S[A]==A_val,S.columns != A]
      common,counts = np.unique(Sv[Sv.keys()[-1]],return_counts=True)
      if len(counts) == 1 or max_depth == 0:
        tree[A][A_val] = common[0]
      else:
        tree[A][A_val] = ID3(max_labels,max_depth-1,variant,Sv,A,tree=None)     
    return tree
    


      


In [16]:
def find_best_split(max_labels,variant,attr,S,total_ent):
  #Need to check the
  
  list_of_IG = {}
  for A1 in attr:
    curr_attr = A1
    curr_df = S[[curr_attr,S.keys()[-1]]]
    
    len_A1_df = len(curr_df[curr_attr])
    found_sum_expected_entropy = 0
    all_attr_values = curr_df[curr_attr].unique()
    #print(all_attr_values)
    #print(counter)
    for A2 in all_attr_values:
      
      attribute_specific = curr_df[curr_df[curr_attr] == A2]
      #print(attribute_specific)
      found_expected_entropy = 0
      entropy = find_total_entropy(max_labels,variant,attribute_specific)
      if (entropy == 1 and variant == 2):
        entropy = 0
      #print(entropy)
      found_sum_expected_entropy += (entropy * (len(attribute_specific)/len_A1_df))
    #print(found_sum_expected_entropy)
    list_of_IG[A1] = total_ent - found_sum_expected_entropy   
  return list_of_IG


In [17]:
def find_gain(total_entropy,expec):
  if variant == 1:
    #use entropy
    d
  if variant == 2:
    d
    #use ME
  if variant == 3:
    d
    #use Gini Index

In [18]:
def find_total_entropy(max_labels,variant,S):
  if variant == 1:
    #use entropy
    type_name = S.keys()[-1]
    entropy = 0
    label_names = S[type_name].unique()
    for val in label_names:
      #print(val)
      p = S[type_name].value_counts()[val] / len(S[type_name])
      #print(p)
      entropy += -p*np.log2(p)
    return entropy
  elif variant == 2:
    counter = len(labels_value)
    
    #use ME
    type_name = S.keys()[-1]  
    #ME = 0
    label_names = S[type_name].unique()
    list_of_prob = []
    for val in label_names:
      #print(S)
      #print(val)
      #print(S[type_name].value_counts()[val])
      p = S[type_name].value_counts()[val] / len(S)
      list_of_prob.append(p)
    #print(list_of_prob)
    if len(list_of_prob) == max_labels:
      #print("Yes")
      return np.min(list_of_prob)
    else:
      #print("No")
      return 0
  elif variant == 3:
    #use gini
    type_name = S.keys()[-1]
    #Gini = 0
    label_names = S[type_name].unique()
    sum_p = 0
    for val in label_names:
      p = S[type_name].value_counts()[val] / len(S[type_name])
      sum_p += np.square(p)

    return 1 - sum_p

In [118]:
train_tree = ID3(-1*float("inf"),1,1,df,"none",tree=None)
test_tree = ID3(-1*float("inf"),1,1,df_test,"none",tree=None)

pprint.pprint(train_tree)
aye = list(dict_path(train_tree,path=None))




{'safety': {'high': {'persons': {'2': 'unacc', '4': 'acc', 'more': 'acc'}},
            'low': 'unacc',
            'med': {'persons': {'2': 'unacc', '4': 'acc', 'more': 'acc'}}}}
(['safety', 'med', 'persons', '4'], 'acc')


In [157]:
print(aye[6])

(['safety', 'low'], 'unacc')


In [188]:
df_hey = df[df.safety == 'med']
print(len(df_hey))
df_hey = df_hey[df.persons == '4']
print(len(df_hey))
df_hey = df_hey[df.label == 'acc']
print(len(df_hey))

344
112
54


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [71]:
def dict_path(my_dict, path=None):
    if path is None:
        path = []
    for k,v in my_dict.items():
        newpath = path + [k]
        if isinstance(v, dict):
            for u in dict_path(v, newpath):
                yield u
        else:
            yield newpath, v

In [186]:
#TRAINING ERROR
# DEPTH 1 -> 6
# RECORD ERROR
list_of_variants = ["Entropy","MajorityError","Gini Index"]
error_dict_per_variant = {}
for b in range(1,4):
  error_dict_per_variant[list_of_variants[b-1]] = {}
  error_counter = 0
  for i in range(1,7):
    train_tree = ID3(-1*float("inf"),i,b,df,"none",tree=None)
    train_tree_path = list(dict_path(train_tree,path=None))

    #print(train_tree_path)

    df_filter = df
    for j in range(len(train_tree_path)):

      curr_path = train_tree_path[j]
      for a in range(0,len(curr_path[0])):
        if (a % 2) == 0:
          #print(curr_path)
          #print(curr_path[0])
          df_filter = df_filter[df_filter[curr_path[0][a]] == curr_path[0][a+1]]
      

      length = len(df_filter)
      count_of_accuracies = len(df_filter[df_filter[df_filter.keys()[-1]] == curr_path[1]]) 
      count_of_inaccuracies = length - count_of_accuracies
      if length == 0:
        error_counter += 0
      else:
        error_counter += count_of_inaccuracies / length
  
  
  error_dict_per_variant[list_of_variants[b-1]] = error_counter / 6    




In [187]:
error_dict_per_variant

{'Entropy': 0.204700328407225,
 'Gini Index': 0.204700328407225,
 'MajorityError': 0.3078020253886749}